# 앙상블(Ensemble)
`06_Ensemble.ipynb`

- 정형데이터(Structured Data) 기준으로는 가장 뛰어난 성과를 내는 알고리즘
- 대부분의 앙상블 학습 -> 트리 기반

> 결정트리를 모으면? 포레스트~
### Random Forest
- 결정트리를 랜덤하게 만들어서 트리의 숲을 만듦
- 각 결정트리의 예측을 종합해 최종 예측을 만듦
- Overfitting에 상대적으로 안전

### 데이터 분할
- (복원 추출) 데이터가 1000개면, 각 트리마다 1000개 데이터를 뽑는데, 이때 중복을 허용
- 노드 분할 시, 분류/회귀의 특성 선택 방식이 다름(분류: 개수를 루트함, 회귀: 특성 전부 다 씀)
- 기본값 100개의 트리를 만들어서
    - 분류: 1등이 살아남음
    - 회귀: 100개 평균냄

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('./wine.csv')
wine.head()
X = wine[['alcohol', 'sugar', 'pH']]
y = wine['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs = -1, random_state=42)

scores = cross_validate(rf, X_train, y_train, return_train_score=True, n_jobs=-1)

#      훈련셋의 일부                      검증셋(훈련셋의 일부)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 전형적인 과대적합
# 왜 과대적합이 일어났을까? 하이퍼 파라미터 설정을 안 해서 그럼
# depth를 설정 안 하면 당연히 과적합 발생.

# 진짜 학습 시키기
rf.fit(X_train, y_train)
# 트리(100개)들의 특성별 중요도 수치화
print(rf.feature_importances_)

"""
노드마다 랜덤하게 특성을 뽑아서 사용하기 때문에 특정 특성에 과도하게 집중되는 것을 방지함.
그렇기 때문에 다양한 특성이 훈련에 쓰일 기회를 주고, 과적합 가능성을 줄임
"""

0.997844759088341 0.8918317274785446
[0.2311695  0.49701637 0.27181413]


'\n노드마다 랜덤하게 특성을 뽑아서 사용하기 때문에 특정 특성에 과도하게 집중되는 것을 방지함.\n그렇기 때문에 다양한 특성이 훈련에 쓰일 기회를 주고, 과적합 가능성을 줄임\n'

In [3]:
# OOB(Out of Bag) 샘플
# 중복허용으로 트리 100개 만듦 -> 쓰지 않은 샘플을 모아서 만든 샘플이 OOB
# 이건 검증샘플같은 역할을 함.

rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(X_train, y_train)
print(rf.score(X_train, y_train), rf.oob_score_)


0.9973316912972086 0.8981937602627258


In [4]:
# 랜덤포레스트 하이퍼파라미터 튜닝
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

rgen = randint(0, 10) # 인스턴스 생성
rgen.rvs(10)

np.unique(rgen.rvs(1000), return_counts=True)

ugen = uniform(0, 1)
ugen.rvs(10)

array([0.61126803, 0.12718689, 0.1102845 , 0.67271954, 0.97575972,
       0.58827879, 0.98474339, 0.62573932, 0.02339364, 0.20738888])

In [ ]:
params = {
    # 노드 분할을 위한 최소 불순도
    'min_impurity_decrease': uniform(0.0001, 0.001),
    # 트리 깊이
    'max_depth': randint(10, 50),
    # 노드를 나누기 위한 최소 샘플 수
    'min_samples_split': randint(2, 25),
    # 리프 노드 개수 최솟값
    'min_samples_leaf': randint(1, 25),
    # (새로 추가된 것)
    # Random Foreset의 HP
    'max_features': ['sqrt', 'log2', None], #  노드별 특성 뽑을 때
    'n_estimators': randint(50, 300),
}

In [8]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(
    RandomForestClassifier(random_state=42), # rf
    params,
    n_iter = 100,
    n_jobs = -1,
    random_state=42
    )

gs.fit(X_train, y_train)
print(gs.best_params_)

{'max_depth': 30, 'min_impurity_decrease': np.float64(0.000290911031150346), 'min_samples_leaf': 1, 'min_samples_split': 4}


In [10]:
gs.best_score_
dt = gs.best_estimator_
print('최적파라미터: ', gs.best_params_)
print('최고 교차검증 점수: ', gs.best_score_)
print(f'최종 테스트 결과: {gs.score(X_test, y_test)}')
best_rf = gs.best_estimator_
print('특성 중요도: ', best_rf.feature_importances_)

최적파라미터:  {'max_depth': 30, 'min_impurity_decrease': np.float64(0.000290911031150346), 'min_samples_leaf': 1, 'min_samples_split': 4}
최고 교차검증 점수:  0.8807442742062866
최종 테스트 결과: 0.8529230769230769
특성 중요도:  [0.16406255 0.59946537 0.23647208]


In [ ]:
# # 시각화 1
# import matplotlib.pyplot as plt

# results = pd.DataFrame(rs.cv_results_)

# # max_depth vs mean_test_score
# plt.figure(figsize=(8, 5))
# plt.plot(results['param_max_depth'], results['mean_test_score'], marker='o')
# plt.xlabel("max_depth")
# plt.ylabel("Mean CV Score")
# plt.title("Effect of max_depth on Model Performance")
# plt.show()


NameError: name 'rs' is not defined

## Extra Tree
- 랜덤 포레스트와 매우 유사
- 부트스트랩 샘플(복원추출)을 사용하지 않음
- 전체 훈련세트 그대로 사용함. 특성도 무작위 선택
- 노드 분할할 때, 최적(불순도/정보이득)을 찾는 것이 아니라, 무작위로 분할
- 성능이 낮아질 수 있지만, 많은 트리를 앙상블 하기 때문에 과대적합을 방지함.
- 데이터 노이즈가 많은 경우에 자주 활용(패턴과 상관 없는 불필요/잘못된 데이터)

### > `확률적 경사 하강법`을 보시라

In [15]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, X_train, y_train, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

et.fit(X_train, y_train)

print(et.score(X_test, y_test), et.feature_importances_)

0.997844759088341 0.8903937240035804
0.8713846153846154 [0.20702369 0.51313261 0.2798437 ]


## Gradient Boosting
- `Boosting`: 약한 모델(결정트리가 얕다)을 여러 개 차례대로 학습.
- 앞에 모델에서 틀린 부분을 뒤에 모델이 보완해주는 방식
- `Gradient`: 오차를 줄이기 위해 경사하강법 아이디어를 적용

#### 장점
- 비선형, 복잡한 데이터에서 예측이 뛰어남
- 과적합 방지 가능한 여러 규제들이 있음
- 별도 전용 라이브러리(XGBoost, LightGBM, CatBoost) -> 대회에서 1등한 적이 많음
#### 단점
- 학습 속도 느림(첫 트리가 안 나오면 다음 트리 작업을 못 함)

1. 약한 첫 번째 결정트리 학습 -> 기본 예측값 생성
2. 오차(Residual) 계산(답 - 예측)
3. 해당 잔차를 예측하도록 또 다른 작은 트리를 학습 -> 예측값 업데이트
4. 반복 -> 오차가 줄어듦

In [25]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
scores = cross_validate(gb, X_train, y_train, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

gb.fit(X_test, y_test)
gb.score(X_test, y_test)

# 사이즈가 작음에도 불구하고 overfitting이 덜 일어남... ㄷㄷ

0.8894704231708938 0.8715107671247301


0.8886153846153846

## 히스토그램 기반 Gradient Boosting
- 입력 특성들을 256개 구간으로 나눔(이 숫자는 고정)
- 노드분할할 때, 구간 경계값을 쓰기 때문에 최적분할을 가장 빠르게 찾을 수 있다.

In [26]:
# from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, X_train, y_train, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# scores: 0.9380129799494501 0.8805410414363187
# 기본 GB 대비 점수는 좀 높아졌음. 걸리는 시간은 확연히 줆

hgb.fit(X_test, y_test)
hgb.score(X_test, y_test)

0.9380129799494501 0.8805410414363187


0.9636923076923077

In [27]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, X_train, y_train, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

xgb.fit(X_train, y_train)
xgb.score(X_train, y_train)

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/jun-seokoh/.pyenv/versions/3.13.2/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <6984A3F0-3899-36C4-A85D-20B5520FF130> /Users/jun-seokoh/.pyenv/versions/3.13.2/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/l

In [29]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, X_train, y_train, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

lgb.fit(X_train, y_train)
lgb.score(X_train, y_train)

OSError: dlopen(/Users/jun-seokoh/.pyenv/versions/3.13.2/lib/python3.13/site-packages/lightgbm/lib/lib_lightgbm.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib
  Referenced from: <D44045CD-B874-3A27-9A61-F131D99AACE4> /Users/jun-seokoh/.pyenv/versions/3.13.2/lib/python3.13/site-packages/lightgbm/lib/lib_lightgbm.dylib
  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file), '/Users/jun-seokoh/.pyenv/versions/3.13.2/lib/libomp.dylib'